<a href="https://colab.research.google.com/github/rvkushnir/project_fifa_players/blob/main/notebooks/01_eda_and_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ініціалізація середовища**

Клонуємо репозиторій з GitHub у /content, переходимо в корінь проєкту.
Це гарантує однакові шляхи для даних/виводів у всіх сесіях Colab.
Після клонування перевіряємо наявність data/raw/fifa_players.csv.
Вихід: робочий каталог → pwd = /content/project_fifa_players.

In [1]:
#  Ініціалізація середовища: клон репозиторію і перехід у робочу теку
REPO = "rvkushnir/project_fifa_players"
BRANCH = "main"
WORKDIR = "/content/project_fifa_players"

# чистий клон
!rm -rf "$WORKDIR"
!git clone --depth 1 -b "$BRANCH" "https://github.com/{REPO}.git" "$WORKDIR"
%cd "$WORKDIR"

# Якщо у репо використовується Git LFS (великі файли) — підтягнемо дані
!git lfs install 2>/dev/null || true
!git lfs pull 2>/dev/null || true

# Перевіримо, що CSV на місці
!ls -lah data/raw


Cloning into '/content/project_fifa_players'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 69 (delta 6), reused 56 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 6.24 MiB | 7.66 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/project_fifa_players
Updated git hooks.
Git LFS initialized.
total 11M
drwxr-xr-x 2 root root 4.0K Sep 16 17:28 .
drwxr-xr-x 4 root root 4.0K Sep 16 17:28 ..
-rw-r--r-- 1 root root  11M Sep 16 17:28 fifa_players.csv
-rw-r--r-- 1 root root   61 Sep 16 17:28 README.md


Мета блоку: зчитати fifa_players.csv, підготувати primary_position, побудувати is_top_value (топ-15%), зберегти кеш Parquet.
Вхід: data/raw/fifa_players.csv.
Вихід: data/interim/players_clean.parquet, out/tables/summary.json.
Ключові рішення: поріг за квантіллю 0.85; пропуски/нульові value відкидаємо.

1. Завантажуємо датасет з гравцями.
2. Проводимо базову валідацію та очищення, створюємо primary_positions, обчислюємо топ гравців по квантилю 0.85.
3. Зберігаємо швидкий кеш як parquet для прискорення наступних кроків.
4. Виводимо стислий начальний аналіз.

In [2]:
import os, re, json
from pathlib import Path
import numpy as np
import pandas as pd

#  Налаштування шляху
DATA_CSV = Path("data/raw/fifa_players.csv")
INTERIM_DIR = Path("data/interim")
OUT_DIR = Path("out/tables")
INTERIM_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

CACHE_TO_PARQUET = True                          # вкл/викл кеш Parquet
PARQUET_PATH = INTERIM_DIR / "players_clean.parquet"
VALUE_TOP_Q = 0.85                               # топ-15% за ціною

#  Допоміжні функції
def make_primary_position(s: pd.Series) -> pd.Series:
    # Беремо перше значення з player_positions як primary
    return s.astype(str).str.split(",", n=1, expand=True)[0].str.strip()

def parse_money_to_eur(x):
    # парсер
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip().replace("€","").replace("$","").replace("£","").replace(" ","").replace(",","")
    m = re.match(r"^([0-9]*\.?[0-9]+)([MKmk])?$", s)
    if m:
        val = float(m.group(1))
        suf = m.group(2)
        if suf:
            if suf.upper()=="M": val *= 1_000_000
            if suf.upper()=="K": val *= 1_000
        return val
    if re.match(r"^[0-9]+(\.[0-9]+)?$", s):
        return float(s)
    return np.nan

#  Завантаження CSV
if not DATA_CSV.exists():
    raise FileNotFoundError(f"Не знайдено файл: {DATA_CSV.resolve()}")

df = pd.read_csv(DATA_CSV, low_memory=False)
print(f"[INFO] Завантажено: {df.shape[0]:,} × {df.shape[1]:,}")
print("[INFO] Колонки (перші 20):", list(df.columns[:20]))

#  Базова підготовка
# Вартість у EUR
value_col = "value_eur" if "value_eur" in df.columns else None
if value_col is None:
    raise ValueError("Не знайдено стовпця з вартістю (value_eur).")

# привід числових колонок до типу float/int, щоб уникати рядкових значень
for col in ["overall", "potential", "wage_eur", "height_cm", "weight_kg"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

df["_value_eur"] = df[value_col].apply(parse_money_to_eur)

# primary_position
if "player_positions" not in df.columns:
    raise ValueError("Не знайдено стовпця player_positions.")
df["primary_position"] = make_primary_position(df["player_positions"])

# Ціль: топ-15% за вартістю
value_threshold = df["_value_eur"].quantile(VALUE_TOP_Q)
df["is_top_value"] = (df["_value_eur"] >= value_threshold).astype(int)

#  Легка діагностика
quant_series = df["_value_eur"].quantile([0.5,0.75,0.9,0.95,VALUE_TOP_Q])
quant_fmt = {f"{q:.2f}": f"{v:,.0f}" for q, v in quant_series.to_dict().items()}
class_balance = df["is_top_value"].value_counts(normalize=True).to_dict()

print("\n[INFO] Квантили вартості (EUR):")
print(json.dumps(quant_fmt, ensure_ascii=False, indent=2))
print(f"[INFO] Поріг топ-15%: {value_threshold:,.0f} EUR")
print("[INFO] Баланс класів:", {int(k): round(v,4) for k,v in class_balance.items()})

if "league_name" in df.columns:
    top_leagues = df["league_name"].value_counts().head(10)
    print("\n[INFO] Топ-10 ліг за чисельністю:")
    print(top_leagues)

if "primary_position" in df.columns:
    top_pos = df["primary_position"].value_counts().head(10)
    print("\n[INFO] Топ-10 позицій:")
    print(top_pos)

#  Фільтрація явних пропусків/нульових вартостей
df_clean = df[df["_value_eur"].notna() & (df["_value_eur"] > 0)].copy()

#  Кеш у Parquet
if CACHE_TO_PARQUET:
    df_clean.to_parquet(PARQUET_PATH, index=False)
    print(f"\n[OK] Збережено кеш: {PARQUET_PATH} → {df_clean.shape[0]:,}×{df_clean.shape[1]:,}")

#  Міні-SUMMARY для наступних кроків
summary = {
    "rows_total": int(df.shape[0]),
    "rows_clean": int(df_clean.shape[0]),
    "cols": int(df.shape[1]),
    "value_top_quantile": float(VALUE_TOP_Q),
    "value_threshold_eur": float(value_threshold),
    "class_balance": {int(k): float(v) for k,v in class_balance.items()}
}
print("\n===== SUMMARY =====")
print(json.dumps(summary, ensure_ascii=False, indent=2))

# збереження summary у файл для зручності перевірки/звіту
(Path(OUT_DIR) / "summary.json").write_text(json.dumps(summary, ensure_ascii=False, indent=2))


[INFO] Завантажено: 19,239 × 80
[INFO] Колонки (перші 20): ['sofifa_id', 'player_url', 'short_name', 'long_name', 'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_position', 'club_team_id', 'club_name', 'league_name', 'league_level', 'club_loaned_from', 'club_joined']

[INFO] Квантили вартості (EUR):
{
  "0.50": "975,000",
  "0.75": "2,000,000",
  "0.90": "5,500,000",
  "0.95": "11,500,000",
  "0.85": "3,300,000"
}
[INFO] Поріг топ-15%: 3,300,000 EUR
[INFO] Баланс класів: {0: 0.8482, 1: 0.1518}

[INFO] Топ-10 ліг за чисельністю:
league_name
USA Major League Soccer          771
Argentina Primera División       727
English League Championship      717
English Premier League           652
Spain Primera Division           633
English League One               626
Spanish Segunda División         625
English League Two               613
Japanese J. League Division 1    589
French Ligue 1                   577
Name: count, dtyp

211